In [98]:
import pandas as pd
from PIL import Image
import random
import albumentations as A
import numpy as np
import json
import os
from tqdm import tqdm

In [99]:
# Define augmentation functions
def rotate_image(image, angle):
    transform = A.Rotate(limit=(angle, angle), p=1.0)
    return transform(image=image)['image']

def gaussian_blur_image(image, sigma):
    transform = A.GaussianBlur(blur_limit=(sigma, sigma), p=1.0)
    return transform(image=image)['image']

def adv_blur_image(image, shift, intensity):
    transform = A.GaussNoise(var_limit=shift, noise_scale_factor=intensity, p=1.0)
    return transform(image=image)['image']

def drop_pixels(image, drop_fraction=0.01):
    height, width, _ = image.shape
    num_pixels_to_drop = int(width * height * drop_fraction)
    
    for _ in range(num_pixels_to_drop):
        x = random.randint(0, width - 1)
        y = random.randint(0, height - 1)
        image[y, x] = [0, 0, 0]  # Drop to black or any other color
    
    return image

In [100]:
dataset_dir = 'E:/datasets'
dataset_name = 'DOTA_dataset_512'
coco_dir = f'{dataset_dir}/{dataset_name}'
eval_coco = f'{coco_dir}/val_coco.json'
output_dir = coco_dir

with open(eval_coco, 'r') as f:
    coco_json = json.load(f)

img_id_to_img_path = {img_dict['id']:img_dict['path'] for img_dict in coco_json['images']}
img_id_to_img_dict = {img_dict['id']:img_dict for img_dict in coco_json['images']}
select_classes = [3,12]
cat_id_to_cat = {cat['id']:cat['name'] for cat in coco_json['categories']}
for sel in select_classes:
    print(f'using {cat_id_to_cat[sel]}')

annot_df = pd.DataFrame(coco_json['annotations'])
plane_ship_df = annot_df[annot_df['category_id'].isin(select_classes)]
print(plane_ship_df['category_id'].unique())
plane_ship_df.head()


using ship
using plane
[ 3 12]


,id,image_id,category_id,bbox,area,iscrowd,segmentation,polygon
696,697,49,3,"[109.0, 453.0, 36.0, 50.0]",916.0,0,"[[145.0, 461.0, 127.0, 453.0, 109.0, 493.0, 12...","POLYGON ((2193 461, 2175 453, 2157 493, 2176 5..."
698,699,50,3,"[299.0, 305.0, 18.0, 30.0]",420.0,0,"[[299.0, 335.0, 315.0, 335.0, 317.0, 305.0, 30...","POLYGON ((2756 335, 2772 335, 2774 305, 2762 3..."
717,718,57,3,"[265.0, 446.0, 32.0, 29.0]",457.0,0,"[[297.0, 460.0, 288.0, 446.0, 265.0, 461.0, 27...","POLYGON ((706 869, 697 855, 674 870, 683 884, ..."
724,725,61,3,"[109.0, 44.0, 36.0, 50.0]",916.0,0,"[[145.0, 52.0, 127.0, 44.0, 109.0, 84.0, 128.0...","POLYGON ((2193 461, 2175 453, 2157 493, 2176 5..."
732,733,66,3,"[336.0, 364.0, 27.0, 19.0]",444.0,0,"[[363.0, 381.0, 362.0, 364.0, 336.0, 366.0, 33...","POLYGON ((4459 790, 4458 773, 4432 775, 4433 7..."


In [101]:
ps_annotations = plane_ship_df.to_dict(orient='records')

In [102]:
print(len(ps_annotations))
print(ps_annotations[0])

28319
{'id': 697, 'image_id': 49, 'category_id': 3, 'bbox': [109.0, 453.0, 36.0, 50.0], 'area': 916.0, 'iscrowd': 0, 'segmentation': [[145.0, 461.0, 127.0, 453.0, 109.0, 493.0, 128.0, 503.0, 145.0, 461.0]], 'polygon': 'POLYGON ((2193 461, 2175 453, 2157 493, 2176 503, 2193 461))'}


In [103]:
subset_json = coco_json.copy()
subset_json['annotations'] = ps_annotations

In [104]:
plane_ship_images = list(plane_ship_df['image_id'].unique())
new_image_dir = f'{output_dir}/aug_images_10_10'
os.makedirs(new_image_dir,exist_ok=True)
new_images = []
for img_id in tqdm(plane_ship_images):
    new_dict = img_id_to_img_dict[img_id].copy()
    img_name = new_dict['file_name']
    new_path = f'{new_image_dir}/{img_name}'
    new_dict['path'] = new_path
    new_images.append(new_dict)

    org_img = Image.open(img_id_to_img_dict[img_id]['path']).convert('RGB')
    img_np = np.array(org_img)
    aug_img = gaussian_blur_image(img_np, 7)
    noise_var = 200
    aug_img = adv_blur_image(aug_img, (noise_var,noise_var), (1))
    aug_img = drop_pixels(aug_img, 0.01)
    aug_img = Image.fromarray(aug_img)
    aug_img.save(new_path)

100%|██████████| 3500/3500 [02:59<00:00, 19.46it/s]


In [105]:
subset_json['images'] = new_images

with open(f'{output_dir}/plane_ship_aug_val_coco.json', 'w') as f:
    json.dump(subset_json, f, indent=4)